In [40]:
import os 

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import IPython.display as ipd

import librosa
import librosa.display
import pickle

import warnings
warnings.filterwarnings('ignore')

### Load Saved Model

In [49]:
rf = pickle.load(open("../Saved_Models/model.obj", "rb"))
encoder = pickle.load(open("../Saved_Models/encoder.obj", "rb"))

### Get Full File Path from user

In [55]:
#"/Users/vedantterkar/Desktop/Proj/*"
audio_file = input("Enter Full Audio File Path: ")

Enter Full Audio File Path: /Users/vedantterkar/Desktop/Proj/C.mp3


### Listen to File

In [56]:
isError = False
try:
    ipd.Audio(audio_file) 
except:
    isError = True
    print("Can't process the file Provided!")

In [57]:
ipd.Audio(audio_file) 

### Extract Features

In [58]:
audio_clips = [audio_file]

m = {}
m["chroma"], m["spec_cnt"], m["spec_bw"], m["roll_off"] = [], [], [], []
m["zcr"] = []

for i in range(20):
    m["mfcc_"+str(i)] = []
try:
    for song in audio_clips:
        x, sr = librosa.load(song, mono=True, offset=0)
        duration = librosa.get_duration(y=x, sr=sr)
        ratio = 30.00 / duration
        m["chroma"].append(np.mean(librosa.feature.chroma_stft(y=x, sr=sr)) * ratio)
        m["spec_cnt"].append(np.mean(librosa.feature.spectral_centroid(y=x, sr=sr)) * ratio)
        m["spec_bw"].append(np.mean(librosa.feature.spectral_bandwidth(y=x, sr=sr)) * ratio)
        m["roll_off"].append(np.mean(librosa.feature.spectral_rolloff(y=x, sr=sr)) * ratio)
        m["zcr"].append(np.mean(librosa.feature.zero_crossing_rate(x)) * ratio)
        mfcc = librosa.feature.mfcc(y=x, sr=sr)
        c = 0
        for e in mfcc:
            m["mfcc_"+str(c)].append(np.mean(e) * ratio)
            c += 1
except:
     print("Can't process the file Provided!")

import pandas as pd
dataFrame = pd.DataFrame(data=m, columns=sorted(list(m.keys()), reverse = True))
dataFrame

,zcr,spec_cnt,spec_bw,roll_off,mfcc_9,mfcc_8,mfcc_7,mfcc_6,mfcc_5,mfcc_4,...,mfcc_16,mfcc_15,mfcc_14,mfcc_13,mfcc_12,mfcc_11,mfcc_10,mfcc_1,mfcc_0,chroma
0,0.171042,2885.288563,2654.354566,5991.632442,-1.416173,-22.196572,8.142225,-26.922613,27.604628,-31.316813,...,-9.257309,-4.159099,-10.335095,-2.535346,-10.304897,-0.53345,-14.865768,120.005567,-112.403638,0.419094


In [59]:
if len(dataFrame) > 0:
    pred = rf.predict(dataFrame)
    print(pred[0], encoder.inverse_transform(pred)[0])
else:
    print("Please Clear the Error Occured earlier!")

9 rock
